| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | **Workflows:** | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb)  | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |

# Towards some ML Applications

This notebook focuses on using geochemistry for some machine learning applications. First we'll look at some mineral chemistry data to see how well we can classify titanium phase polymorphs and whether these are located with mineralization.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pyrolite

np.random.seed(13)  # set a random seed, so everyone gets the same results..

## Rutile Geochemistry

Included in this repository is a cleaned version of titanium phase minor + trace element abundances an appendix table from Plavsa et al. (2018), which we'll use here for an initial foray into using machine learning to help with some geological problems. This dataset was collected with addressing potential polymorph-based partitioning effects on mineralization indicators such that titanium phases across a broader range of rock types might be able to be compared. For more, see the paper:

Plavsa, D., Reddy, S. M., Agangi, A., Clark, C., Kylander-Clark, A., & Tiddy, C. J. (2018). Microstructural, trace element and geochronological characterization of TiO2 polymorphs and implications for mineral exploration. Chemical Geology, 476, 130–149. https://doi.org/10.1016/j.chemgeo.2017.11.011


On loading this paper, we can see that it is a dataset of reasonable but manageable size which consists of minor and trace elements (all in ppm), and columns for sample ID, phase ID and a binary indicator for whether the phase came from a mineralized system or not:

In [ ]:
df = pd.read_csv("../data/rutile/Plavsa2018.csv").set_index("Grain ID", drop=True)
df.columns

### Some Minor Cleaning

There are a few 'bdl' items for Hf within this table causing the column to show up as 'object' instead of a numerical type, which can complicate numerical analysis. For our purposes we can remove them by converting all the element data to numerical values:

In [ ]:
df.dtypes

In [ ]:
df.pyrochem.elements = df.pyrochem.elements.apply(
    pd.to_numeric, errors="coerce"
).astype(float)

In [ ]:
df.dtypes

### Exploratory Data Analysis

To get an idea of how samples, phases and mineralization patterns might differ, we do a bit of exploratory data analysis, starting with some spider diagrams to see how patterns might look across all of these elements:

In [ ]:
from pyrolite.plot.color import process_color
from pyrolite.util.plot.legend import proxy_line


def add_categorical_spider_legend(ax, categories, **kwargs):
    ax.legend(
        [
            proxy_line(c=c, marker="D", **kwargs)
            for c in process_color(c=categories.unique())["c"]
        ],
        categories.unique(),
    )

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
df.pyrochem.elements.pyroplot.spider(c=df["Sample ID"], ax=ax, alpha=0.5)
add_categorical_spider_legend(ax, df["Sample ID"])

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
ax = df.pyrochem.elements.pyroplot.spider(c=df["Phase ID"], ax=ax, alpha=0.5)
add_categorical_spider_legend(ax, df["Phase ID"])

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
df.pyrochem.elements.pyroplot.spider(c=df["Mineralized"], ax=ax, alpha=0.5)
add_categorical_spider_legend(ax, df["Mineralized"])

## Putting some Numbers on the patterns

We can clearly see a few key points early on - rutile is enriched in all minor elements other than Al relative to the other two phases which are otherwise trickier to pull apart (perhaps with Al, Fe). Mineralized and unmineralised titanium phases seem to show distinct patterns, especially for Sn, Cr, +/- Ta-W (take this with a grain of salt, noting we only have five individual samples in the dataset - it's still a useful first example). 
To get a more numerical overview of how these classes differ, we can use some of the inbuilt `pandas` functionality to get some overview statistics based on these breakdowns:

In [ ]:
df.pyrochem.elements.groupby(df["Phase ID"]).agg("mean")

We can add extra items in the chain (a log transform) and some styling to `pandas` output too!

In [ ]:
df.pyrochem.elements.apply(np.log).groupby(df["Phase ID"]).agg(
    "mean"
).style.background_gradient(axis=0, cmap="cividis")

We can also use multiple aggregation metrics:

In [ ]:
df.pyrochem.elements.groupby(df.Mineralized).agg(["mean", "std"]).T

## Building a Classification Model

With the quick detour into exploration complete for now, we can have a look at building some models which can i) automatically pick up on the patterns which are easy to see by eye above, ii) potentially pick up on some other subtleties and iii) be able to make predictions about new samples we might aquire down the line where we don't know the specific phase (e.g. hitting Ti hotspots in a grain mount under LAICPMS) or we don't know whether a system is mineralize (or, both!). The type of models we're working with below are generally referred to as predictive models, which are typically constructed more for making predictions than for inference/understanding the structure of a dataset.

First, let's have a look at how well we can distinguish phases using the minor+trace element dataset. The key thing to notice, especially for these types of datasets, is the frequency of the most common label - this gives you a 'baseline' to compare to. Imagine if the model just predicted the most frequent phase all of the time - this would be the success rate (this is one reason imbalanced datasets can be hard to work with). Let's look a the proportions of phases in our dataset:

In [ ]:
df["Phase ID"].value_counts() / df.index.size

So, at worst we should be able to score 45% without trying.

In our predictive modeling problem we have a dataset which we want to use to predict another variable (or, target). This falls in the realm of *supervised learning*, which delineates structure in a dataset which relates to specific labels or values (e.g. classification, regression problems), as opposed to working from unlabeled data and attempting to create or delineate structure (e.g. clustering). In the simpler regression sense, this means we need an input `X` and a target `y`, and we're trying to construct a model `f` such that `f(X)` $\approx$ `y`.

In our case, the `X` is our elemental data, and the `y` or target our polymorph labels:

In [ ]:
X = df.pyrochem.elements.apply(pd.to_numeric, errors="coerce")
y = df["Phase ID"]

Note that the models below aren't necessarily going to deal with non numeric values (or NULL values) well, so we should make sure we get rid of them early:

In [ ]:
fltr = ~(pd.isnull(X).sum(axis=1) > 0)
X, y = X.loc[fltr, :], y.loc[fltr]

We can see that we should still have 95% of our data, which is good to know:

In [ ]:
fltr.sum() / fltr.size

When it comes to predictive modeling using tabular data in Python, a key library worth getting acquainted with is [`scikit-learn`](https://scikit-learn.org/stable/index.html) (or `sklearn` as an import).

Here we're going to import a specific classifer model class (there are many, check them out in the `scikit-learn` docs!), `sklearn.ensemble.RandomForestClassifier` which is a [random forest model](https://en.wikipedia.org/wiki/Random_forest).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier

In order for everyone to get the same results, we can set the random state, which governs non-deterministic behaviour (random effects):

In [ ]:
random_state = 27
classifier = RandomForestClassifier(random_state=random_state)

First we want to fit our classifier model to our input data and labels:

In [ ]:
classifier.fit(X, y)

Let's make some predictions on our dataset and see how well we do:

In [ ]:
predictions = classifier.predict(X)
predictions[:10]

We can check these against what we know the phases to be by sticking them both in a table:

In [ ]:
df.loc[fltr, ["Phase ID"]].assign(Predictions=predictions)

Or simply calling score on the model:

In [ ]:
classifier.score(X, y)

It looks pretty good, but this wasn't really a fair test - we're using the same data to train the model as we are to examine it; this is referred to as 'data leakage' in ML modeling. Instead what we want to do is keep back a hold-out-set for testing the model which we don't use for training the model. `scikit-learn` has some built in tools for this, thankfully. These allow us to specify the proportion of data we keep for testing, and whether we 'stratify' the dataset such that we have roughly equal proportions of labels in our training and testing sets (in this case, that's probably a good idea):

In [ ]:
from sklearn.model_selection import train_test_split

XX_train, XX_test, yy_train, yy_test = train_test_split(X, y, stratify=y, test_size=0.3)

In [ ]:
XX_train.index.size, XX_test.index.size

In [ ]:
classifier = RandomForestClassifier(random_state=random_state).fit(XX_train, yy_train)

We don't do quite as well here, but this is closer to what it might look like in the real world (assuming the things we find are similar to those in our training set, of course..):

In [ ]:
classifier.score(XX_test, yy_test)

We can also chain `scikit-learn` components together to make a 'pipeline', adding additional preprocessing steps or bringing together/splitting parts of our dataset:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from pyrolite.util.skl.transform import LogTransform

transform = LogTransform()  # log-transform the data
clf = RandomForestClassifier(random_state=random_state)  # our classifer model
pipe = make_pipeline(transform, clf)
pipe

In this case it won't help us much (scaling isn't necessarily needed for random forests, and we have a fairly high overall classification accuracy), but it's good to keep in mind:

In [ ]:
pipe.fit(XX_train, yy_train).score(XX_test, yy_test)

Another way to look at how well a classifier model performs is on a per-class basis, such as that used in confusion matrix. In this instance is shows us some new information we wouldn't necessarily have seen otherwise - that our predictions for brookite are the worst, and it's mostly because it's getting misclassified as anatase (~4% of the time):

In [ ]:
from pyrolite.util.skl.vis import plot_confusion_matrix

fig, ax = plt.subplots(1, figsize=(5, 4))

plot_confusion_matrix(pipe, XX_test, yy_test, ax=ax, normalize=True)

ax.set_title("Polymorph Classifier\nConfusion Matrix")
ax.set(xlabel="Predicted Polymorph", ylabel="True Polymorph");

## Mineralisation Classifier

We can do the same for mineralization which we've done for polymorphs:

In [ ]:
X = df.pyrochem.elements.apply(pd.to_numeric, errors="coerce")
y = df["Mineralized"]

fltr = ~(pd.isnull(X).sum(axis=1) > 0)
X, y = X.loc[fltr, :], y.loc[fltr]

XX_train, XX_test, yy_train, yy_test = train_test_split(X, y, stratify=y, test_size=0.3)

Our baseline here is 71%:

In [ ]:
y.value_counts() / y.index.size

Which we can easily surpass uisng the same pipeline we created above:

In [ ]:
pipe.fit(XX_train, yy_train).score(XX_test, yy_test)

Given we're working with random forests, a handy thing to do at this stage might be to look at the relative feature importances, which can be accessed from the 'model'/estimator part of the pipeline, which happens to be the last bit:

In [ ]:
pipe.steps[-1]

As might be expected, Sn tops the list:

In [ ]:
pd.Series(pipe.steps[-1][1].feature_importances_, index=X.columns).sort_values(
    ascending=False
)

This is where we'll leave this example - there's likely more we could do to make it peform in the real world, but for the purpose here it looks pretty good! Given the initial intention of the paper, what could be done is constructing a model to identify polymorphs, which is then fed in as an additional input to a mineralization classifier.

Often the best place to start in improving models is cleaning up your data (it's rare that you'll start with super-clean data). To go beyond this, I'd suggest first having a look at playing with different models, then hyperparameter optimization - changing the internal parameters of our workflow components to optimally achieve our desired objective. `scikit-learn` has some built in tools for this too, of course!

---

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | **Workflows:** | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb)  | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |